In [ ]:
from IPython.display import display
import pandas as pd

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
import json

with open("data/export_dunkerque_entreprôt.json", "r") as f:
    report_dk = json.load(f)

In [ ]:
data_dunkerque = {"Dunkerque": report_dk}

In [ ]:
VegaLite({
  "title": "Répartition des imports/exports de Dunkerque en 1789 par produits (SITC simpl)",
  "data": {"values": report_dk},
  "mark": {"type":"bar", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {"field": "value", "type": "quantitative", "title": "Valeur (Lt)"},
    "column": {"field": "group"},
    "y": {"field": "product", "sort": "-x", "title": "Produits"},
    "color": {"field": "partner"}
  }
})

In [ ]:
def pyramide_port_franc(port_franc, data):
    report = data[port_franc]
    VegaLite({
      "title": "Répartition des imports/exports de {} en 1789 par produits (SITC simpl)".format(port_franc),
      "data": {"values": report},
      "spacing": 0,
      "hconcat": [{
        "transform": [{
          "filter": {"field": "group", "equal": "import"}
        }],
        "title": "import",
        "mark": {"type":"bar", "tooltip": {"content": "data"}},
        "encoding": {
          "y": {
            "field": "product", "title": None, "sort": "-x"
          },
          "x": {
            "field": "value",
            "type": "quantitative",
            "title": "Valeur (Lt)",
              "sort": "descending"
          },
          "color": {
            "field": "partner",
          }
        }
      },
      {
        "transform": [{
          "filter": {"field": "group", "equal": "export"}
        }],
        "title": "export",
        "mark": {"type":"bar", "tooltip": {"content": "data"}},
        "encoding": {
          "y": {
            "field": "product", "sort": "-x", "title": None, "axis": None

          },
          "x": {
            "field": "value",
            "type": "quantitative",
            "title": "Valeur (Lt)",
            "sort": "ascending"
          },
          "color": {
            "field": "partner",
          }
        }
      }
      ]
    }
    )

In [ ]:
pyramide_port_franc("Dunkerque", data_dunkerque)